
# The Structural Factor Analysis of benchmark for Over-Refusal Behavior Based on Varies LLMS

In [1]:
import importlib
import re
import math
import logging
import gc
from pathlib import Path
from functools import reduce
from collections import Counter
from statistics import mean

import numpy as np
import pandas as pd
import altair as alt
from tqdm import tqdm
from IPython.display import display

# --- 全局配置 ---
alt.data_transformers.disable_max_rows()
alt.renderers.enable("default")

# 定义特征缓存路径
CACHE_DIR = Path("./feature_cache")
CACHE_DIR.mkdir(exist_ok=True) # 确保缓存目录存在

# --- 0. Stanza Setup & Utilities (已启用 CUDA 加速) ---

def _imp(name):
    """Safely import a module, raising an error if not found."""
    try:
        return importlib.import_module(name)
    except Exception as e:
        print(f"[WARN]: Required library '{name}' not found. Please install it (e.g., pip install {name}).")
        raise

try:
    stanza = _imp("stanza")
    torch = _imp("torch")
except Exception:
    print("[FATAL]: Stanza or PyTorch failed to load. Linguistic feature computation will fail.")
    stanza = None
    torch = None

# 检查 CUDA 可用性
USE_CUDA = False
if torch and torch.cuda.is_available():
    USE_CUDA = True
    print(f"[INFO]: CUDA is available. Using GPU for Stanza processing. Device: {torch.cuda.get_device_name(0)}")
elif torch:
    print("[INFO]: CUDA is not available. Stanza will run on CPU.")

# Cache for Stanza pipelines to avoid re-loading
_NLP_CACHE = {}

def get_nlp(lang_code: str):
    """Gets a cached Stanza pipeline for a given language code."""
    if stanza is None:
        raise RuntimeError("Stanza library is not available.")

    if lang_code not in _NLP_CACHE:
        print(f"Loading Stanza pipeline for language: {lang_code} ...")
        try:
            _NLP_CACHE[lang_code] = stanza.Pipeline(
                lang_code, processors='tokenize,pos,lemma,depparse',
                tokenize_no_ssplit=False, use_gpu=USE_CUDA,
                logging_level='WARN'
            )
        except Exception:
            print(f"Downloading Stanza model for language: {lang_code} ...")
            stanza.download(lang_code)
            _NLP_CACHE[lang_code] = stanza.Pipeline(
                lang_code, processors='tokenize,pos,lemma,depparse',
                tokenize_no_ssplit=False, use_gpu=USE_CUDA,
                logging_level='WARN'
            )
        print(f"Stanza pipeline for {lang_code} loaded.")
    return _NLP_CACHE[lang_code]


# --- 1. Data Loading & Preparation Functions (不变) ---

def get_column_map(df, model_name):
    # ... (函数体不变) ...
    def find_col(suffix_regex, model_name, specific_suffix):
        for c in df.columns:
            if re.search(suffix_regex, c, flags=re.I):
                return c

        fallback = f"{model_name}_{specific_suffix}"
        if fallback in df.columns:
            return fallback

        if model_name == "deepseekv32" and specific_suffix == "result_cn":
            if "deepseekv32_resultcn" in df.columns:
                return "deepseekv32_resultcn"

        return None

    col_map = {
        "TEXT_EN": find_col(r"_result_en$", model_name, "result_en"),
        "TEXT_CN": find_col(r"_result_cn$", model_name, "result_cn"),
        "TEXT_MIX": find_col(r"_result_mix$", model_name, "result_mix"),
        "LABEL_EN": "Final_Label_EN" if "Final_Label_EN" in df.columns else None,
        "LABEL_CN": "Final_Label_CN" if "Final_Label_CN" in df.columns else None,
        "LABEL_MIX": "Final_Label_MIX" if "Final_Label_MIX" in df.columns else None,
    }

    if not any([col_map["TEXT_EN"], col_map["TEXT_CN"], col_map["TEXT_MIX"]]):
        raise ValueError(f"No result columns found for model '{model_name}'. Searched for suffixes _result_en, _result_cn, _result_mix.")

    return col_map

def load_and_prep_data(csv_path, model_name):
    # ... (函数体不变) ...
    if not Path(csv_path).exists():
        raise FileNotFoundError(f"CSV file not found at: {csv_path}")

    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]

    col_map = get_column_map(df, model_name)

    for lab in [col_map["LABEL_EN"], col_map["LABEL_CN"], col_map["LABEL_MIX"]]:
        if lab is not None and lab in df.columns:
            df[lab] = df[lab].astype(str).str.lower().str.strip()

    if "id" not in df.columns:
        df = df.reset_index().rename(columns={"index": "id"})

    rename_map = {"Rewrite Method": "method", "Category": "category"}
    df = df.rename(columns=rename_map)

    print("--- Column Mapping Found ---")
    print(f"   English Prompts: {col_map['TEXT_EN']}")
    print(f"   Chinese Prompts: {col_map['TEXT_CN']}")
    print(f"   Mixed Prompts: {col_map['TEXT_MIX']}")
    print(f"   Labels: EN={col_map['LABEL_EN']}, CN={col_map['LABEL_CN']}, MIX={col_map['LABEL_MIX']}")

    keep = ["id"]
    if "category" in df.columns: keep.append("category")
    if "method" in df.columns: keep.append("method")

    for c in col_map.values():
        if c is not None and c not in keep:
            keep.append(c)

    df_prepped = df.loc[:, list(dict.fromkeys(keep))].copy()

    return df_prepped, col_map


# --- 2. Stanza Feature Engineering Functions (加入缓存和 model_name 参数) ---

CN_COMPLEX_PUNCT = re.compile(r"[；：——…—]")
SUBORDINATE_TAGS = {"mark", "advcl", "acl", "ccomp", "xcomp", "dep", "parataxis"}

def count_complex_punct(text: str) -> int:
    return len(CN_COMPLEX_PUNCT.findall(str(text)))

def unigram_entropy(tokens):
    if not tokens:
        return 0.0
    cnt = Counter(tokens)
    n = len(tokens)
    ent = 0.0
    for c in cnt.values():
        p = c / n
        ent -= p * math.log(p + 1e-12)
    return float(ent)

def type_token_ratio(tokens):
    return (len(set(tokens)) / len(tokens)) if tokens else 0.0

def compute_dep_tree_depth(sent):
    children = {}
    for w in sent.words:
        children.setdefault(w.head, []).append(w.id)
    def dfs(node_id, depth):
        if node_id not in children:
            return depth
        return max((dfs(ch, depth + 1) for ch in children[node_id]), default=depth)
    depths = [dfs(ch, 1) for ch in children.get(0, [])] or [1]
    return max(depths)

def compute_dep_distance_mean(sent):
    if not sent.words:
        return 0.0
    dists = [abs(w.id - w.head) for w in sent.words if w.head is not None]
    return mean(dists) if dists else 0.0

def compute_sub_clause_count(sent):
    return sum(1 for w in sent.words if (w.deprel or '').lower() in SUBORDINATE_TAGS)

def stanza_features_for_text(text: str, nlp):
    text = str(text or "").strip()
    if not text:
        return {
            "character_len": 0, "prompt_count": 0, "token_len": 0,
            "dep_depth_mean": 0.0, "dep_distance_mean": 0.0,
            "sub_clause_count": 0, "punct_complex_count": 0,
            "type_token_ratio": 0.0, "lexical_information_entropy": 0.0
        }

    try:
        doc = nlp(text)
        sents = doc.sentences
        sent_count = len(sents)
        tok_len = sum(len(s.words) for s in sents)

        dep_depths = [compute_dep_tree_depth(s) for s in sents] if sents else [0]
        dep_depth_mean = mean(dep_depths) if dep_depths else 0.0

        dep_distance_means = [compute_dep_distance_mean(s) for s in sents] if sents else [0.0]
        dep_distance_mean = mean(dep_distance_means) if dep_distance_means else 0.0

        sub_clause_total = sum(compute_sub_clause_count(s) for s in sents)
        tokens = [w.text for s in sents for w in s.words]

        return {
            "character_len": len(text),
            "prompt_count": sent_count,
            "token_len": tok_len,
            "dep_depth_mean": float(dep_depth_mean),
            "dep_distance_mean": float(dep_distance_mean),
            "sub_clause_count": int(sub_clause_total),
            "punct_complex_count": int(count_complex_punct(text)),
            "type_token_ratio": float(type_token_ratio(tokens)),
            "lexical_information_entropy": float(unigram_entropy(tokens)),
        }
    except Exception as e:
        logging.warning(f"Stanza failed to process text: {text[:50]}... Error: {e}")
        return {
            "character_len": len(text), "prompt_count": 0, "token_len": 0,
            "dep_depth_mean": 0.0, "dep_distance_mean": 0.0,
            "sub_clause_count": 0, "punct_complex_count": 0,
            "type_token_ratio": 0.0, "lexical_information_entropy": 0.0
        }


def compute_features(df_prepped, column_map, model_name):
    """
    Computes Stanza features for all text variants, with added caching.
    """
    # --- Caching Check ---
    cache_path = CACHE_DIR / f"{model_name}_features.pkl"
    if cache_path.exists():
        print(f">> [CACHE HIT] Loading features for {model_name} from {cache_path}...")
        return pd.read_pickle(cache_path)

    print(f">> [CACHE MISS] Computing features for {model_name}...")
    # --- End Caching Check ---

    if "id" not in df_prepped.columns:
        df_prepped = df_prepped.reset_index().rename(columns={"index": "id"})

    VARIANTS = []
    if column_map["TEXT_EN"]:
        VARIANTS.append(("EN", column_map["TEXT_EN"], column_map["LABEL_EN"], "en"))
    if column_map["TEXT_CN"]:
        VARIANTS.append(("CN", column_map["TEXT_CN"], column_map["LABEL_CN"], "zh"))
    if column_map["TEXT_MIX"]:
        VARIANTS.append(("MIX", column_map["TEXT_MIX"], column_map["LABEL_MIX"], "zh"))

    if not VARIANTS:
        raise ValueError("No text/label variants available to compute features.")

    feature_frames = []

    label_cols_to_merge = ["id"]
    for lab in column_map.values():
        if lab is not None and "Label" in lab and lab in df_prepped.columns and lab not in label_cols_to_merge:
            label_cols_to_merge.append(lab)

    base_df = df_prepped[label_cols_to_merge].copy()
    base_df["id"] = pd.to_numeric(base_df["id"], errors="coerce").astype("Int64")
    feature_frames.append(base_df)

    for name, text_col, label_col, lang_code in VARIANTS:
        print(f">> Computing features for {name} using Stanza lang='{lang_code}' ...")
        nlp = get_nlp(lang_code)

        rows = []
        for _id, text in tqdm(df_prepped[["id", text_col]].itertuples(index=False, name=None), total=len(df_prepped)):
            feats = stanza_features_for_text(text, nlp)
            rows.append({
                "id": _id,
                f"dep_depth_mean_{name}": feats["dep_depth_mean"],
                f"entropy_token_{name}": feats["lexical_information_entropy"],
            })

        df_f = pd.DataFrame(rows)
        df_f["id"] = pd.to_numeric(df_f["id"], errors="coerce").astype("Int64")
        feature_frames.append(df_f)

    df_feat = reduce(lambda a, b: a.merge(b, on="id", how="left"), feature_frames)

    # --- Save Cache ---
    df_feat.to_pickle(cache_path)
    print(f">> Features computed and saved to {cache_path}.")
    # --- End Save Cache ---

    return df_feat


# --- 3. Plotting Functions (不变) ---

# ... (所有 plotting functions 不变) ...
def prepare_unified_data(all_feature_dfs):
    long_frames = []

    FEATURE_MAP = {
        "dep_depth_mean": "Avg Dependency Tree Depth",
        "entropy_token": "Vocabulary Information Entropy",
    }

    for model_name_raw, df in all_feature_dfs.items():
        model_name = df['model_title_name'].iloc[0] if 'model_title_name' in df.columns else model_name_raw

        feature_cols = [c for c in df.columns if re.search(r'_(EN|CN|MIX)$', c) and 'Final_Label' not in c and 'model_title_name' not in c]

        keep_cols = ['id']
        for name in ['EN', 'CN', 'MIX']:
             label_col_name = f'Final_Label_{name}'
             if label_col_name in df.columns:
                 df[f'Label_{name}'] = df[label_col_name].astype(str).str.lower().str.strip()
                 keep_cols.append(f'Label_{name}')

        if not feature_cols:
            continue

        df_features = df.melt(
            id_vars=keep_cols,
            value_vars=feature_cols,
            var_name='Feature_Lang',
            value_name='Value'
        )

        df_features[['Feature_Raw', 'Language']] = df_features['Feature_Lang'].str.rsplit('_', n=1, expand=True)
        df_features['Model'] = model_name

        df_features['Label'] = df_features.apply(
            lambda row: row[f'Label_{row["Language"]}'] if f'Label_{row["Language"]}' in row else None, axis=1
        )

        df_features['Feature'] = df_features['Feature_Raw'].map(FEATURE_MAP)
        df_features = df_features[
            (df_features['Label'].isin(['answer', 'refuse'])) &
            (df_features['Feature'].notna())
        ].copy()

        long_frames.append(df_features[['Model', 'Language', 'Feature', 'Value', 'Label']])

    return pd.concat(long_frames, ignore_index=True)


def create_unified_comparison_charts(df_unified):

    if df_unified.empty:
        return alt.Chart(pd.DataFrame({'text': ['No unified data to display']})).mark_text()

    max_dep = df_unified[df_unified['Feature'] == 'Avg Dependency Tree Depth']['Value'].max()
    max_ent = df_unified[df_unified['Feature'] == 'Vocabulary Information Entropy']['Value'].max()

    domain_y_dep = [0, max_dep * 1.05 if max_dep else 1]
    domain_y_ent = [0, max_ent * 1.05 if max_ent else 1]

    base = alt.Chart(df_unified).encode(
        x=alt.X('Model:N', title=None, axis=alt.Axis(labels=True, labelAngle=-45)),
        color=alt.Color('Label:N', title='Behavior',
                        scale=alt.Scale(domain=['answer', 'refuse'], range=['#377eb8', '#e41a1c'])),
        tooltip=[
            'Model:N',
            'Feature:N',
            'Language:N',
            'Label:N',
            alt.Tooltip('mean(Value):Q', title='Mean Value', format='.2f'),
            alt.Tooltip('Value:Q', title='Distribution (Violin)')
        ]
    ).properties(
        width=250,
        height=200
    )

    violin = base.transform_density(
        'Value',
        as_=['Value', 'Density'],
        groupby=['Model', 'Feature', 'Language', 'Label'],
        extent='min-max',
        steps=200
    ).mark_area(orient='vertical').encode(
        y=alt.Y('Value:Q', title='Value', scale=alt.Scale(type='linear', domain=alt.FieldRange('domain_y'))),
        x=alt.X('Density:Q', stack='center', title='Density', axis=None),
        order=alt.Order('Label', sort='descending')
    ).transform_calculate(
        domain_y=f'datum.Feature === "Avg Dependency Tree Depth" ? {domain_y_dep} : {domain_y_ent}'
    ).facet(
        column=alt.Column('Feature:N', header=alt.Header(titleOrient="bottom", labelOrient="bottom")),
        row=alt.Row('Language:N', header=alt.Header(titleOrient="right", labelOrient="right")),
    ).properties(
        title=alt.TitleParams(
            text="Unified Feature Comparison by Model and Refusal Behavior (Violin Plots)",
            anchor="middle",
            orient="top",
            dy=-10,
            fontSize=16
        )
    ).resolve_scale(
        y='independent'
    )

    return violin


# --- 6. Main Configuration and Analysis Loop (已优化显存清理和缓存调用) ---

# Configuration
DATA_DIR = Path("../data/label_fusion") # Base directory for CSVs - 请根据您的文件路径修改此路径

ANALYSIS_CONFIG = [
    {
        "model_name": "gemma34b",
        "title_name": "Gemma-34b",
        "csv_path": DATA_DIR / "test_gemma34b_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "llama318b",
        "title_name": "Llama-318b",
        "csv_path": DATA_DIR / "test_llama318b_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "qwen34b",
        "title_name": "Qwen-34b",
        "csv_path": DATA_DIR / "test_qwen34b_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "gemini25flash",
        "title_name": "Gemini-25flash",
        "csv_path": DATA_DIR / "test_gemini25flash_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "deepseekv32",
        "title_name": "Deepseek-V32",
        "csv_path": DATA_DIR / "test_deepseekv32_on_local_data_results_labeled.csv",
    },
]

# Analysis Loop
all_feature_dfs = {}

# 1. Iterate through models and compute features
for config in ANALYSIS_CONFIG:
    model_key = config['model_name']
    print(f"\n=======================================================")
    print(f"Processing Model: {config['title_name']}")
    print(f"=======================================================")

    # 1. Load and Prepare Data
    try:
        df_prepped, col_map = load_and_prep_data(
            config["csv_path"],
            model_key
        )
    except Exception as e:
        print(f"[ERROR] Failed to load data for {config['title_name']}: {e}")
        continue

    # 2. Compute Linguistic Features (使用缓存和 model_key)
    try:
        df_feat = compute_features(df_prepped, col_map, model_key)
        df_feat['model_title_name'] = config['title_name']
        all_feature_dfs[model_key] = df_feat
    except Exception as e:
        print(f"[ERROR] Failed to compute features for {config['title_name']}: {e}")
        continue

    # 3. 显存清理：处理完一个模型后，立即清理显存
    if USE_CUDA:
        print("[INFO]: Clearing Stanza cache and GPU memory...")
        _NLP_CACHE.clear()
        torch.cuda.empty_cache()
        gc.collect() # 强制运行垃圾回收
        print("[INFO]: Memory cleared for next model.")

print("\n\n--- Individual Analysis Complete. Generating Unified Charts ---")

# 4. Generate Unified Charts
try:
    df_unified = prepare_unified_data(all_feature_dfs)
    unified_chart = create_unified_comparison_charts(df_unified)

    print("\nDisplaying Unified Comparison Charts (Violin Plots):")
    display(unified_chart)

except Exception as e:
    print(f"[FATAL ERROR] Failed to generate unified charts: {e}")

print("\n\n--- All Analysis Complete ---")

d:\Env\anaconda\envs\capstone\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO]: CUDA is available. Using GPU for Stanza processing. Device: NVIDIA RTX 2000 Ada Generation Laptop GPU

Processing Model: Gemma-34b
--- Column Mapping Found ---
   English Prompts: gemma34b_result_en
   Chinese Prompts: gemma34b_result_cn
   Mixed Prompts: gemma34b_result_mix
   Labels: EN=Final_Label_EN, CN=Final_Label_CN, MIX=Final_Label_MIX
>> [CACHE MISS] Computing features for gemma34b...
>> Computing features for EN using Stanza lang='en' ...
Loading Stanza pipeline for language: en ...


2025-11-09 17:12:53 WARNING: Language en package default expects mwt, which has been added


Stanza pipeline for en loaded.


100%|██████████| 600/600 [03:19<00:00,  3.00it/s]


...

>> Computing features for CN using Stanza lang='zh' ...
Loading Stanza pipeline for language: zh ...
Stanza pipeline for zh loaded.


100%|██████████| 600/600 [02:27<00:00,  4.08it/s]


>> Computing features for MIX using Stanza lang='zh' ...


100%|██████████| 600/600 [03:02<00:00,  3.28it/s]


>> Features computed and saved to feature_cache\deepseekv32_features.pkl.
[INFO]: Clearing Stanza cache and GPU memory...
[INFO]: Memory cleared for next model.


--- Individual Analysis Complete. Generating Unified Charts ---
[FATAL ERROR] Failed to generate unified charts: 'min-max' is an invalid value for `extent`. Valid values are of type 'array'.


--- All Analysis Complete ---


In [14]:
import importlib
import re
import math
import logging
import gc
from pathlib import Path
from functools import reduce
from collections import Counter
from statistics import mean

import numpy as np
import pandas as pd
import altair as alt
from tqdm import tqdm
from IPython.display import display

# --- 全局配置 ---
alt.data_transformers.disable_max_rows()

# 尝试启用最兼容的渲染器以确保在 Jupyter 环境中正确显示
try:
    alt.renderers.enable('notebook')
except Exception:
    try:
        alt.renderers.enable('jupyterlab')
    except Exception:
        alt.renderers.enable('default')

print("[INFO]: Altair renderer configured.")


# 定义特征缓存路径
CACHE_DIR = Path("./feature_cache")
CACHE_DIR.mkdir(exist_ok=True) # 确保缓存目录存在

# --- 0. Stanza Setup & Utilities (已启用 CUDA 加速) ---

def _imp(name):
    """Safely import a module, raising an error if not found."""
    try:
        return importlib.import_module(name)
    except Exception as e:
        print(f"[WARN]: Required library '{name}' not found. Please install it (e.g., pip install {name}).")
        raise

try:
    stanza = _imp("stanza")
    torch = _imp("torch")
except Exception:
    print("[FATAL]: Stanza or PyTorch failed to load. Linguistic feature computation will fail.")
    stanza = None
    torch = None

# 检查 CUDA 可用性
USE_CUDA = False
if torch and torch.cuda.is_available():
    USE_CUDA = True
    print(f"[INFO]: CUDA is available. Using GPU for Stanza processing. Device: {torch.cuda.get_device_name(0)}")
elif torch:
    print("[INFO]: CUDA is not available. Stanza will run on CPU.")

# Cache for Stanza pipelines to avoid re-loading
_NLP_CACHE = {}

def get_nlp(lang_code: str):
    """Gets a cached Stanza pipeline for a given language code."""
    if stanza is None:
        raise RuntimeError("Stanza library is not available.")

    if lang_code not in _NLP_CACHE:
        print(f"Loading Stanza pipeline for language: {lang_code} ...")
        try:
            _NLP_CACHE[lang_code] = stanza.Pipeline(
                lang_code, processors='tokenize,pos,lemma,depparse',
                tokenize_no_ssplit=False, use_gpu=USE_CUDA,
                logging_level='WARN'
            )
        except Exception:
            print(f"Downloading Stanza model for language: {lang_code} ...")
            stanza.download(lang_code)
            _NLP_CACHE[lang_code] = stanza.Pipeline(
                lang_code, processors='tokenize,pos,lemma,depparse',
                tokenize_no_ssplit=False, use_gpu=USE_CUDA,
                logging_level='WARN'
            )
        print(f"Stanza pipeline for {lang_code} loaded.")
    return _NLP_CACHE[lang_code]


# --- 1. Data Loading & Preparation Functions ---

def get_column_map(df, model_name):
    def find_col(suffix_regex, model_name, specific_suffix):
        for c in df.columns:
            if re.search(suffix_regex, c, flags=re.I):
                return c

        fallback = f"{model_name}_{specific_suffix}"
        if fallback in df.columns:
            return fallback

        if model_name == "deepseekv32" and specific_suffix == "result_cn":
            if "deepseekv32_resultcn" in df.columns:
                return "deepseekv32_resultcn"

        return None

    col_map = {
        "TEXT_EN": find_col(r"_result_en$", model_name, "result_en"),
        "TEXT_CN": find_col(r"_result_cn$", model_name, "result_cn"),
        "TEXT_MIX": find_col(r"_result_mix$", model_name, "result_mix"),
        "LABEL_EN": "Final_Label_EN" if "Final_Label_EN" in df.columns else None,
        "LABEL_CN": "Final_Label_CN" if "Final_Label_CN" in df.columns else None,
        "LABEL_MIX": "Final_Label_MIX" if "Final_Label_MIX" in df.columns else None,
    }

    if not any([col_map["TEXT_EN"], col_map["TEXT_CN"], col_map["TEXT_MIX"]]):
        raise ValueError(f"No result columns found for model '{model_name}'. Searched for suffixes _result_en, _result_cn, _result_mix.")

    return col_map

def load_and_prep_data(csv_path, model_name):
    if not Path(csv_path).exists():
        raise FileNotFoundError(f"CSV file not found at: {csv_path}")

    df = pd.read_csv(csv_path)
    df.columns = [c.strip() for c in df.columns]

    col_map = get_column_map(df, model_name)

    for lab in [col_map["LABEL_EN"], col_map["LABEL_CN"], col_map["LABEL_MIX"]]:
        if lab is not None and lab in df.columns:
            df[lab] = df[lab].astype(str).str.lower().str.strip()

    if "id" not in df.columns:
        df = df.reset_index().rename(columns={"index": "id"})

    rename_map = {"Rewrite Method": "method", "Category": "category"}
    df = df.rename(columns=rename_map)

    print("--- Column Mapping Found ---")
    print(f"   English Prompts: {col_map['TEXT_EN']}")
    print(f"   Chinese Prompts: {col_map['TEXT_CN']}")
    print(f"   Mixed Prompts: {col_map['TEXT_MIX']}")
    print(f"   Labels: EN={col_map['LABEL_EN']}, CN={col_map['LABEL_CN']}, MIX={col_map['LABEL_MIX']}")

    keep = ["id"]
    if "category" in df.columns: keep.append("category")
    if "method" in df.columns: keep.append("method")

    for c in col_map.values():
        if c is not None and c not in keep:
            keep.append(c)

    df_prepped = df.loc[:, list(dict.fromkeys(keep))].copy()

    return df_prepped, col_map


# --- 2. Stanza Feature Engineering Functions ---

CN_COMPLEX_PUNCT = re.compile(r"[；：——…—]")
SUBORDINATE_TAGS = {"mark", "advcl", "acl", "ccomp", "xcomp", "dep", "parataxis"}

def count_complex_punct(text: str) -> int:
    return len(CN_COMPLEX_PUNCT.findall(str(text)))

def unigram_entropy(tokens):
    if not tokens:
        return 0.0
    cnt = Counter(tokens)
    n = len(tokens)
    ent = 0.0
    for c in cnt.values():
        p = c / n
        ent -= p * math.log(p + 1e-12)
    return float(ent)

def type_token_ratio(tokens):
    return (len(set(tokens)) / len(tokens)) if tokens else 0.0

def compute_dep_tree_depth(sent):
    children = {}
    for w in sent.words:
        children.setdefault(w.head, []).append(w.id)
    def dfs(node_id, depth):
        if node_id not in children:
            return depth
        return max((dfs(ch, depth + 1) for ch in children[node_id]), default=depth)
    depths = [dfs(ch, 1) for ch in children.get(0, [])] or [1]
    return max(depths)

def compute_dep_distance_mean(sent):
    if not sent.words:
        return 0.0
    dists = [abs(w.id - w.head) for w in sent.words if w.head is not None]
    return mean(dists) if dists else 0.0

def compute_sub_clause_count(sent):
    return sum(1 for w in sent.words if (w.deprel or '').lower() in SUBORDINATE_TAGS)

def stanza_features_for_text(text: str, nlp):
    text = str(text or "").strip()
    if not text:
        return {
            "character_len": 0, "prompt_count": 0, "token_len": 0,
            "dep_depth_mean": 0.0, "dep_distance_mean": 0.0,
            "sub_clause_count": 0, "punct_complex_count": 0,
            "type_token_ratio": 0.0, "lexical_information_entropy": 0.0
        }

    try:
        doc = nlp(text)
        sents = doc.sentences
        sent_count = len(sents)
        tok_len = sum(len(s.words) for s in sents)

        dep_depths = [compute_dep_tree_depth(s) for s in sents] if sents else [0]
        dep_depth_mean = mean(dep_depths) if dep_depths else 0.0

        dep_distance_means = [compute_dep_distance_mean(s) for s in sents] if sents else [0.0]
        dep_distance_mean = mean(dep_distance_means) if dep_distance_means else 0.0

        sub_clause_total = sum(compute_sub_clause_count(s) for s in sents)
        tokens = [w.text for s in sents for w in s.words]

        return {
            "character_len": len(text),
            "prompt_count": sent_count,
            "token_len": tok_len,
            "dep_depth_mean": float(dep_depth_mean),
            "dep_distance_mean": float(dep_distance_mean),
            "sub_clause_count": int(sub_clause_total),
            "punct_complex_count": int(count_complex_punct(text)),
            "type_token_ratio": float(type_token_ratio(tokens)),
            "lexical_information_entropy": float(unigram_entropy(tokens)),
        }
    except Exception as e:
        logging.warning(f"Stanza failed to process text: {text[:50]}... Error: {e}")
        return {
            "character_len": len(text), "prompt_count": 0, "token_len": 0,
            "dep_depth_mean": 0.0, "dep_distance_mean": 0.0,
            "sub_clause_count": 0, "punct_complex_count": 0,
            "type_token_ratio": 0.0, "lexical_information_entropy": 0.0
        }


def compute_features(df_prepped, column_map, model_name):
    """
    Computes Stanza features for all text variants, with added caching.
    """
    # --- Caching Check ---
    cache_path = CACHE_DIR / f"{model_name}_features.pkl"
    if cache_path.exists():
        print(f">> [CACHE HIT] Loading features for {model_name} from {cache_path}...")
        return pd.read_pickle(cache_path)

    print(f">> [CACHE MISS] Computing features for {model_name}...")
    # --- End Caching Check ---

    if "id" not in df_prepped.columns:
        df_prepped = df_prepped.reset_index().rename(columns={"index": "id"})

    VARIANTS = []
    if column_map["TEXT_EN"]:
        VARIANTS.append(("EN", column_map["TEXT_EN"], column_map["LABEL_EN"], "en"))
    if column_map["TEXT_CN"]:
        VARIANTS.append(("CN", column_map["TEXT_CN"], column_map["LABEL_CN"], "zh"))
    if column_map["TEXT_MIX"]:
        VARIANTS.append(("MIX", column_map["TEXT_MIX"], column_map["LABEL_MIX"], "zh"))

    if not VARIANTS:
        raise ValueError("No text/label variants available to compute features.")

    feature_frames = []

    label_cols_to_merge = ["id"]
    for lab in column_map.values():
        if lab is not None and "Label" in lab and lab in df_prepped.columns and lab not in label_cols_to_merge:
            label_cols_to_merge.append(lab)

    base_df = df_prepped[label_cols_to_merge].copy()
    base_df["id"] = pd.to_numeric(base_df["id"], errors="coerce").astype("Int64")
    feature_frames.append(base_df)

    for name, text_col, label_col, lang_code in VARIANTS:
        print(f">> Computing features for {name} using Stanza lang='{lang_code}' ...")
        nlp = get_nlp(lang_code)

        rows = []
        for _id, text in tqdm(df_prepped[["id", text_col]].itertuples(index=False, name=None), total=len(df_prepped)):
            feats = stanza_features_for_text(text, nlp)
            rows.append({
                "id": _id,
                f"dep_depth_mean_{name}": feats["dep_depth_mean"],
                f"entropy_token_{name}": feats["lexical_information_entropy"],
            })

        df_f = pd.DataFrame(rows)
        df_f["id"] = pd.to_numeric(df_f["id"], errors="coerce").astype("Int64")
        feature_frames.append(df_f)

    df_feat = reduce(lambda a, b: a.merge(b, on="id", how="left"), feature_frames)

    # --- Save Cache ---
    df_feat.to_pickle(cache_path)
    print(f">> Features computed and saved to {cache_path}.")
    # --- End Save Cache ---

    return df_feat


# --- 3. Plotting Functions ---

def prepare_unified_data(all_feature_dfs):
    long_frames = []

    FEATURE_MAP = {
        "dep_depth_mean": "Avg Dependency Tree Depth",
        "entropy_token": "Vocabulary Information Entropy",
    }

    for model_name_raw, df in all_feature_dfs.items():
        model_name = df['model_title_name'].iloc[0] if 'model_title_name' in df.columns else model_name_raw

        feature_cols = [c for c in df.columns if re.search(r'_(EN|CN|MIX)$', c) and 'Final_Label' not in c and 'model_title_name' not in c]

        keep_cols = ['id']
        for name in ['EN', 'CN', 'MIX']:
             label_col_name = f'Final_Label_{name}'
             if label_col_name in df.columns:
                 df[f'Label_{name}'] = df[label_col_name].astype(str).str.lower().str.strip()
                 keep_cols.append(f'Label_{name}')

        if not feature_cols:
            continue

        df_features = df.melt(
            id_vars=keep_cols,
            value_vars=feature_cols,
            var_name='Feature_Lang',
            value_name='Value'
        )

        df_features[['Feature_Raw', 'Language']] = df_features['Feature_Lang'].str.rsplit('_', n=1, expand=True)
        df_features['Model'] = model_name

        df_features['Label'] = df_features.apply(
            lambda row: row[f'Label_{row["Language"]}'] if f'Label_{row["Language"]}' in row else None, axis=1
        )

        df_features['Feature'] = df_features['Feature_Raw'].map(FEATURE_MAP)
        df_features = df_features[
            (df_features['Label'].isin(['answer', 'refuse'])) &
            (df_features['Feature'].notna())
        ].copy()

        long_frames.append(df_features[['Model', 'Language', 'Feature', 'Value', 'Label']])

    return pd.concat(long_frames, ignore_index=True)


def create_unified_comparison_charts(df_unified):

    if df_unified.empty:
        return alt.Chart(pd.DataFrame({'text': ['No unified data to display']})).mark_text()

    base = alt.Chart(df_unified).encode(
        x=alt.X('Model:N', title=None, axis=alt.Axis(labels=True, labelAngle=-45)),
        color=alt.Color('Label:N', title='Behavior',
                        scale=alt.Scale(domain=['answer', 'refuse'], range=['#377eb8', '#e41a1c'])),
        tooltip=[
            'Model:N',
            'Feature:N',
            'Language:N',
            'Label:N',
            alt.Tooltip('mean(Value):Q', title='Mean Value', format='.2f'),
            alt.Tooltip('Value:Q', title='Distribution (Violin)')
        ]
    ).properties(
        width=250,
        height=200
    )

    # 修复: 移除动态 domain 计算和设置，依赖 resolve_scale 实现独立Y轴
    violin = base.transform_density(
        'Value',
        as_=['Value', 'Density'],
        groupby=['Model', 'Feature', 'Language', 'Label'],
        steps=200
    ).mark_area(orient='vertical').encode(
        # 简化 Y 轴编码，移除冲突的 scale(domain=...) 设置
        y=alt.Y('Value:Q', title='Value'),
        x=alt.X('Density:Q', stack='center', title='Density', axis=None),
        order=alt.Order('Label', sort='descending')
    ).facet(
        column=alt.Column('Feature:N', header=alt.Header(titleOrient="bottom", labelOrient="bottom")),
        row=alt.Row('Language:N', header=alt.Header(titleOrient="right", labelOrient="right")),
    ).properties(
        title=alt.TitleParams(
            text="Unified Feature Comparison by Model and Refusal Behavior (Violin Plots)",
            anchor="middle",
            orient="top",
            dy=-10,
            fontSize=16
        )
    ).resolve_scale(
        y='independent' # 确保每个分面都有独立的 Y 轴
    )

    return violin

# --- 4. Main Configuration and Analysis Loop ---

# Configuration
DATA_DIR = Path("../data/label_fusion") # Base directory for CSVs - 请根据您的文件路径修改此路径

ANALYSIS_CONFIG = [
    {
        "model_name": "gemma34b",
        "title_name": "Gemma-34b",
        "csv_path": DATA_DIR / "test_gemma34b_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "llama318b",
        "title_name": "Llama-318b",
        "csv_path": DATA_DIR / "test_llama318b_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "qwen34b",
        "title_name": "Qwen-34b",
        "csv_path": DATA_DIR / "test_qwen34b_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "gemini25flash",
        "title_name": "Gemini-25flash",
        "csv_path": DATA_DIR / "test_gemini25flash_on_local_data_results_labeled.csv",
    },
    {
        "model_name": "deepseekv32",
        "title_name": "Deepseek-V32",
        "csv_path": DATA_DIR / "test_deepseekv32_on_local_data_results_labeled.csv",
    },
]

# Analysis Loop
all_feature_dfs = {}

# 1. Iterate through models and compute features
for config in ANALYSIS_CONFIG:
    model_key = config['model_name']
    print(f"\n=======================================================")
    print(f"Processing Model: {config['title_name']}")
    print(f"=======================================================")

    # 1. Load and Prepare Data
    try:
        df_prepped, col_map = load_and_prep_data(
            config["csv_path"],
            model_key
        )
    except Exception as e:
        print(f"[ERROR] Failed to load data for {config['title_name']}: {e}")
        continue

    # 2. Compute Linguistic Features (使用缓存和 model_key)
    try:
        df_feat = compute_features(df_prepped, col_map, model_key)
        df_feat['model_title_name'] = config['title_name']
        all_feature_dfs[model_key] = df_feat
    except Exception as e:
        print(f"[ERROR] Failed to compute features for {config['title_name']}: {e}")
        continue

    # 3. 显存清理：处理完一个模型后，立即清理显存
    if USE_CUDA:
        print("[INFO]: Clearing Stanza cache and GPU memory...")
        _NLP_CACHE.clear()
        torch.cuda.empty_cache()
        gc.collect()
        print("[INFO]: Memory cleared for next model.")

print("\n\n--- Individual Analysis Complete. Generating Unified Charts ---")

# 4. Generate Unified Charts
try:
    df_unified = prepare_unified_data(all_feature_dfs)
    unified_chart = create_unified_comparison_charts(df_unified)

    print("\nDisplaying Unified Comparison Charts (Violin Plots):")
    display(unified_chart)

    # 额外的保存操作，方便查看
    unified_chart.save("unified_comparison_chart.html")
    print("\n图表已保存为 unified_comparison_chart.html，您可以在浏览器中打开查看。")

except Exception as e:
    print(f"[FATAL ERROR] Failed to generate unified charts: {e}")

print("\n\n--- All Analysis Complete ---")

[INFO]: Altair renderer configured.
[INFO]: CUDA is not available. Stanza will run on CPU.

Processing Model: Gemma-34b
--- Column Mapping Found ---
   English Prompts: gemma34b_result_en
   Chinese Prompts: gemma34b_result_cn
   Mixed Prompts: gemma34b_result_mix
   Labels: EN=Final_Label_EN, CN=Final_Label_CN, MIX=Final_Label_MIX
>> [CACHE HIT] Loading features for gemma34b from feature_cache/gemma34b_features.pkl...

Processing Model: Llama-318b
--- Column Mapping Found ---
   English Prompts: llama318b_result_en
   Chinese Prompts: llama318b_result_cn
   Mixed Prompts: llama318b_result_mix
   Labels: EN=Final_Label_EN, CN=Final_Label_CN, MIX=Final_Label_MIX
>> [CACHE HIT] Loading features for llama318b from feature_cache/llama318b_features.pkl...

Processing Model: Qwen-34b
--- Column Mapping Found ---
   English Prompts: qwen34b_result_en
   Chinese Prompts: qwen34b_result_cn
   Mixed Prompts: qwen34b_result_mix
   Labels: EN=Final_Label_EN, CN=Final_Label_CN, MIX=Final_Label_MIX


<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting



图表已保存为 unified_comparison_chart.html，您可以在浏览器中打开查看。


--- All Analysis Complete ---


## Model 1: Gemma3-4b

## Model 2: Llama3.1-8b

## Model 3: Qwen3-4b

## Model 4: Gemini-2.5-flash

## Model 5: Deepseek-V3.2